In [197]:
import numpy as np

from bokeh.layouts import column, row
from bokeh.models import CustomJS, WheelZoomTool, Slider, Legend
from bokeh.plotting import ColumnDataSource, figure, output_file, show
from bokeh.io import output_notebook
output_notebook(hide_banner =  True)

In [77]:
x = np.linspace(0, 10, 500)
y = np.sin(x)

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(y_range=(-10, 10), plot_width=400, plot_height=400, toolbar_location=None)

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

amp_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Amplitude")
freq_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Frequency")
phase_slider = Slider(start=0, end=6.4, value=0, step=.1, title="Phase")
offset_slider = Slider(start=-5, end=5, value=0, step=.1, title="Offset")

callback = CustomJS(args=dict(source=source, amp=amp_slider, freq=freq_slider, phase=phase_slider, offset=offset_slider),
                    code="""
    const data = source.data;
    const A = amp.value;
    const k = freq.value;
    const phi = phase.value;
    const B = offset.value;
    const x = data['x']
    const y = data['y']
    for (var i = 0; i < x.length; i++) {
        y[i] = B + A*Math.sin(k*x[i]+phi);
    }
    source.change.emit();
""")

amp_slider.js_on_change('value', callback)
freq_slider.js_on_change('value', callback)
phase_slider.js_on_change('value', callback)
offset_slider.js_on_change('value', callback)

layout = row(
    plot,
    column(amp_slider, freq_slider, phase_slider, offset_slider),
)

#output_file("slider.html", title="slider.py example")

show(layout)

In [78]:
position[399]

array([25.78110997, 10.14694838])

In [79]:
# Two dimensional simulation of one Brownian particle
max_steps = 401
position = np.zeros([max_steps, 2]) # initial position (0,0)
sigma = 1
for t in range(max_steps - 1):
    position[t+1] = position[t] + sigma * np.random.normal(0,1,2)

xposition = position[:,0]
yposition = position[:,1]

minval = np.min(position)
maxval = np.max(position)

source = ColumnDataSource(data=dict(x=xposition, y=yposition))
original = ColumnDataSource(data=dict(x=xposition, y=yposition))

source_dot = ColumnDataSource(data=dict(x=[xposition[-1]], y=[yposition[-1]]))


plot = figure(x_range=(minval, maxval), y_range=(minval, maxval), plot_width=400, 
              plot_height=400, toolbar_location=None)

plot.line('x', 'y', source=source, line_width=2, line_alpha=0.6, legend_label="Brownian trajectory")

plot.circle('x', 'y', source=source_dot, size = 10, color="red")

time_slider = Slider(start=0, end=max_steps-1, value=max_steps-1, step=1, title="Time")

callback = CustomJS(args=dict(source=source, original=original, source_dot=source_dot, time=time_slider),
                    code="""
    const data = source.data;
    const data_dot = source_dot.data;
    const original_data = original.data
    const t = time.value;
    const x = data['x']
    const y = data['y']
    const xtrue = original_data['x']
    const ytrue = original_data['y']
    const xdot = data_dot['x']
    const ydot = data_dot['y']
    for (var i = 0; i < x.length; i++) {
        if (i < t) {
            x[i] = xtrue[i]
            y[i] = ytrue[i]
        } else {
            x[i] = xtrue[t]
            y[i] = ytrue[t]
        }
    }
    xdot[0] = xtrue[t] 
    ydot[0] = ytrue[t] 
    source.change.emit();
    source_dot.change.emit();
""")

time_slider.js_on_change('value', callback)

layout = row(
    plot,
    column(time_slider),
)

show(layout)

## Histogram test

In [115]:
x0 = 0 # initial position for all particles
sigma = 1
max_steps = 250
num_particles = 500

# Function to integrate the trajetory of one Brownian particle in one dimension (x0 initial position)
def integrate_brownian(xinitial):
    xlist=[xinitial]
    for i in range(max_steps):
        xcurrent = xlist[i] 
        xnew = xcurrent + sigma * np.random.normal(0,1)
        xlist.append(xnew)
    return np.array(xlist)

# Integrate Brownian trajetory of all Brownian particles and saves all the data into
# brownian_trajectories (calls integrate_brownian)
brownian_trajectories = []
for i in range(num_particles):
    brownian_trajectory = integrate_brownian(x0)
    brownian_trajectories.append(brownian_trajectory)

# Function to extract positions of all the Brownian particles at a given time step    
def extract_positions(timestep):
    positions = []
    for i in range(num_particles):
        positions.append(brownian_trajectories[i][timestep])
    return positions

# Function to plot the histogram of the particles distribution at a given time step
def plot_particles_distribution(timestep):
    positions = extract_positions(timestep)
    plt.hist(positions, 30, density=True, label="Distribution of Brownian particles")
    plt.xlim([-50,50])
    plt.ylim([0,0.1])
    plt.legend()

In [190]:
len(hist)

30

In [118]:
plot = figure(x_range=(-50,50),plot_width=400,plot_height=400, tools="pan,wheel_zoom,reset")
plot.title.text = '2D Brownian trajectory'
plot.title.text_font_size = '12pt'
plot.toolbar.active_scroll = plot.select_one(WheelZoomTool)
plot.toolbar.logo = None

D = 1/2
time = 1

measured = extract_positions(time)
hist, edges = np.histogram(measured, density=True, bins=30)

x = np.linspace(-50, 50, 100)
pdf = 1/np.sqrt(4 * np.pi * D * time) * np.exp(-x*x/(4 * D * time))

plot.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")
plot.line(x, pdf, line_color="#D95B43", line_width=8, alpha=0.7, legend_label="PDF")

plot.xaxis.axis_label = 'x'
plot.yaxis.axis_label = 'Pr(x)'

show(plot)

In [203]:
D = 1/2

xdomain = np.linspace(-50, 50, 100)
hist_list = []
leftedges_list = []
rightedges_list = []
pdf_list = []
for i in range(max_steps):
    time = i+1
    positions_particles = extract_positions(time)
    hist, edges = np.histogram(positions_particles, density=True, bins=30)
    hist_list.append(hist)
    leftedges_list.append(edges[:-1])
    rightedges_list.append(edges[1:])
    pdf = 1/np.sqrt(4 * np.pi * D * time) * np.exp(-xdomain*xdomain/(4 * D * time))
    pdf_list.append(pdf)


sourceHist = ColumnDataSource(data=dict(hist_data= hist_list[max_steps-1], 
                                        leftedges_data = leftedges_list[max_steps-1],
                                        rightedges_data = rightedges_list[max_steps-1]))
sourcePDF = ColumnDataSource(data=dict(x=xdomain, pdf_data=pdf_list[max_steps-1]))
sourceMain = ColumnDataSource(data=dict(hist_data= hist_list, 
                                        leftedges_data = leftedges_list,
                                        rightedges_data = rightedges_list,
                                        pdf_data = pdf_list))



plot = figure(x_range=(-50,50), y_range=(0,0.2), plot_width=400, plot_height=400, tools="pan,wheel_zoom,reset")
plot.title.text = '2D Brownian trajectory'
plot.title.text_font_size = '12pt'
plot.toolbar.active_scroll = plot.select_one(WheelZoomTool)
plot.toolbar.logo = None

histogram = plot.quad(top='hist_data', bottom=0, left='leftedges_data', 
                      right='rightedges_data', source = sourceHist, alpha=0.6)
distribution = plot.line('x', 'pdf_data', source = sourcePDF, line_color="orange", line_width=4, alpha=0.7)

legend = Legend(items=[('Distribution of Brownian particles', [histogram]), 
                       ('Diffusion equation solution', [distribution])])
plot.add_layout(legend)
legend.click_policy="hide"

#plot.xaxis.axis_label = 'x'
#plot.yaxis.axis_label = 'Pr(x)'

time_slider = Slider(start=1, end=max_steps, value=max_steps, step=1, title="Time")

callback = CustomJS(args=dict(sourceHist=sourceHist, sourcePDF=sourcePDF, sourceMain=sourceMain, time=time_slider),
                    code="""
    const dataHist = sourceHist.data;
    const dataPDF = sourcePDF.data;
    const dataMain = sourceMain.data;
    
    const t = time.value - 1;
    const x = dataPDF['x'];
    const pdf = dataPDF['pdf_data'];
    const hist_data = dataHist['hist_data'];
    const leftedges_data = dataHist['leftedges_data'];
    const rightedges_data = dataHist['rightedges_data'];
    
    const pdf_main = dataMain['pdf_data'];
    const hist_data_main = dataMain['hist_data'];
    const leftedges_data_main = dataMain['leftedges_data'];
    const rightedges_data_main = dataMain['rightedges_data'];

    
    for (var i = 0; i < x.length; i++) {
        pdf[i] = pdf_main[t][i]
    }
    for (var i = 0; i < hist_data.length; i++) {
        hist_data[i] = hist_data_main[t][i];
        leftedges_data[i] = leftedges_data_main[t][i];
        rightedges_data[i] = rightedges_data_main[t][i];
    }
        
    sourceHist.change.emit();
    sourcePDF.change.emit();
""")

time_slider.js_on_change('value', callback)

layout = row(
    plot,
    column(time_slider),
)

show(layout)

pdf = pdf_main[t];
hist_data = hist_data_main[t];
leftedges_data = leftedges_data_main[t];
rightedges_data = rightedges_data_main[t];